In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data_pluto = pd.read_csv("./pluto_23v1.csv")
data_rodent = pd.read_csv("./Rodent_Inspection.csv")

pluto_cols_needed = [
                    "borough",
                    "block",
                    "lot",
                    "cd",
                    "overlay1",
                    "spdist1",
                    "ltdheight",
                    "splitzone",
                    "bldgclass",
                    "landuse",
                    "ownertype",
                    "lotarea",
                    "bldgarea",
                    "comarea",
                    "resarea",
                    "officearea",
                    "retailarea",
                    "garagearea",
                    "strgearea",
                    "factryarea",
                    "numbldgs",
                    "numfloors",
                    "unitsres",
                    "unitstotal",
                    "lotfront",
                    "lotdepth",
                    "ext",
                    "proxcode",
                    "irrlotcode",
                    "lottype",
                    "bsmtcode",
                    "assessland",
                    "assesstot",
                    "exempttot",
                    "yearbuilt",
                    "yearalter1",
                    "yearalter2",
                    "histdist",
                    "landmark",
                    "builtfar",
                    "residfar",
                    "commfar",
                    "facilfar",
                    "borocode",
                    "bbl",
                    "xcoord",
                    "ycoord",
                    "latitude",
                    "longitude"
                    ]
data_pluto = data_pluto[pluto_cols_needed]
print(data_pluto.shape, data_rodent.shape)

/var/folders/6r/skmhybnj3t93yvs97cn0mrrc0000gn/T/ipykernel_83782/3540119670.py:1: DtypeWarning: Columns (21,22,24,26,66) have mixed types. Specify dtype option on import or set low_memory=False.
  data_pluto = pd.read_csv("./pluto_23v1.csv")


(859068, 49) (2354285, 20)


In [4]:
# rodent = data_rodent[(data_rodent["BBL"] < 6e9) & (data_rodent["BBL"]>=1e9)]
# pluto = data_pluto[(data_pluto["bbl"] < 6e9) & (data_pluto["bbl"]>=1e9)]

In [5]:
rodent = data_rodent.copy()
pluto = data_pluto.copy()
print("pluto len: {}, rodent len:{}".format(len(pluto), len(rodent)))

pluto len: 859068, rodent len:2354285


In [6]:
rodent["BORO_CODE"].value_counts()

1    748192
2    690597
3    649584
4    205565
5     60084
9       263
Name: BORO_CODE, dtype: int64

In [7]:
pluto["borocode"].value_counts()

4    324675
3    276869
5    125220
2     89617
1     42687
Name: borocode, dtype: int64

In [8]:
pluto["bbl"].isnull().sum()

0

In [9]:
rodent["BBL"].isnull().sum()

1

In [10]:
len(pluto), pluto["bbl"].nunique()

(859068, 859068)

In [11]:
len(rodent), rodent["BBL"].nunique()

(2354285, 322467)

In [12]:
2358719 / 322588

7.311862189542079

In [13]:
len(pluto.drop_duplicates())

859068

In [14]:
len(rodent.drop_duplicates())

2354285

In [15]:
len(set(rodent["BBL"]) & set(pluto["bbl"]))

313768

In [16]:
rodent = rodent[~rodent["BBL"].isnull()]
rodent.loc[(rodent["BORO_CODE"]==9), "BORO_CODE"] = np.nan
rodent = rodent[rodent["BLOCK"].astype(str).str.len()<=5]



In [17]:
pd.to_datetime("2018 - 07 - 07 04 - PM", format='%Y - %m - %d %I - %p')

Timestamp('2018-07-07 16:00:00')

In [18]:
rodent["INSPECTION_DATE"] = pd.to_datetime(rodent["INSPECTION_DATE"], format="%m/%d/%Y %I:%M:%S %p")
rodent["APPROVED_DATE"] = pd.to_datetime(rodent["APPROVED_DATE"], format="%m/%d/%Y %I:%M:%S %p")

In [19]:
idx_wrong_dt = rodent[rodent["INSPECTION_DATE"] >  rodent["APPROVED_DATE"]].index
rodent.loc[idx_wrong_dt, "APPROVED_DATE"] = pd.NaT

In [20]:
date_processing = "2023-04-23"
idx_wrong_dt2 = rodent[rodent["INSPECTION_DATE"] >= date_processing].index

rodent.loc[idx_wrong_dt2, "INSPECTION_DATE"] = pd.NaT
rodent.loc[idx_wrong_dt2, "APPROVED_DATE"] = pd.NaT

In [21]:
merged = pd.merge(pluto, rodent, left_on="bbl", right_on="BBL", how="inner")

In [151]:
#merged.to_csv("./pluto_rodent_merged.csv")

In [154]:
# example = pd.DataFrame({1:[1,np.nan,3,4,5], 
#                         2:[2,3,4,5,6],
#                         3:[3,4,np.nan,6,7],
#                         4:[4,5,6,7,8]})
# example

# example[[1,2,3,4]].max(axis=1)

,1,2,3,4
0,1.0,2,3.0,4
1,NaN,3,4.0,5
2,3.0,4,NaN,6
3,4.0,5,6.0,7
4,5.0,6,7.0,8


In [25]:
#all BORO CODE are the same
(merged["BORO_CODE"] != merged["borocode"]).sum()

0

In [26]:
merged = merged.drop('BORO_CODE', axis=1)

In [27]:
#all BLOCK are the same
(merged["BLOCK"] != merged["block"]).sum()

0

In [28]:
#all LOT are the same
(merged["LOT"] != merged["lot"]).sum()

0

In [29]:
merged = merged.drop('BLOCK', axis=1)
merged = merged.drop('LOT', axis=1)

In [30]:
merged.to_csv("./pluto_rodent_merged.csv")